<a href="https://colab.research.google.com/github/Kanyinsola-O/CE888/blob/main/Lab%203/bank_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
%matplotlib inline
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import SGDRegressor, BayesianRidge
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import accuracy_score as acc
from sklearn.metrics import make_scorer
from sklearn.dummy import DummyRegressor, DummyClassifier
from sklearn.preprocessing import LabelEncoder

In [ ]:
df=pd.read_csv("https://github.com/albanda/CE888/blob/master/lab3/bank-additional-full.csv", delimiter=";" )
print(df)

In [11]:
print(df.columns)
features = []

print (df[features].head())

In [ ]:
print("Number of rows before dropping NaNs: %d" % len(df))
df = df.dropna()
print("Number of rows after dropping NaNs: %d" % len(df))

In [ ]:
outcomes= []


df[outcomes].head()

In [ ]:

df[["Type"]] = df[["Type"]].apply(LabelEncoder().fit_transform)

df.head()

In [ ]:
#df = df[df.apply(lambda x: np.abs(x - x.mean()) / x.std() < 3).all(axis=1)]

outcomes_of_interest = []

X_df = df[features].copy()
y_df = df[outcomes_of_interest].copy()

cat_features = []
#print(X_df.head())
#print(set(X_df["Category"]))
#print(pd.get_dummies(X_df, columns=["Category"]).head())

X_df = pd.get_dummies(X_df, columns=cat_features)

print(X_df.head()[[]].to_latex())

In [ ]:
y_df[''] = range(1, len(df) + 1)  # add a column
sns_plot = sns.lmplot(x="", y="", data=y_df, fit_reg=False, aspect=2)

# sns_plot.savefig("scaterplot_lpc.pdf", bbox_inches='tight')

In [ ]:
sns_plot = sns.jointplot(x="", y="", data=y_df, ratio = 2)

In [ ]:
X = X_df.values
y = y_df.values.T[0]
y = (y - y.min())/(y.max() - y.min())  # standardise the target variable

In [ ]:
sns_plot = sns.histplot(y, kde=False)
# sns_plot.savefig("histogram_lpc.pdf",bbox_inches='tight')

In [ ]:
clf = DecisionTreeRegressor()
clf.fit(X, y)
print("MSE of the model = %1.4g" % mse(y, clf.predict(X)))

In [ ]:
n_test = 100
n_repeat = 1000

estimator = DecisionTreeRegressor()
#estimator = RandomForestRegressor()

# Compute predictions
y_predicts = np.ones((n_repeat, len(X))) * np.nan

for i in range(n_repeat):
    sample  = np.random.choice(range(len(X)), replace=True, size=len(X))  # bootstrap!
    train_ids = sample[:-n_test]
    test_ids  = sample[-n_test:]
    test_ids = np.setdiff1d(test_ids, train_ids)  # only use those that weren't in the training set for testing
    if len(test_ids) == 0:
        continue
    X_train, y_train = X[train_ids], y[train_ids]
    X_test, y_test = X[test_ids], y[test_ids]
    
    estimator.fit(X_train, y_train)
    y_predict = estimator.predict(X_test)
    y_predicts[i, test_ids] = y_predict

In [ ]:
y_bias = (y - np.nanmean(y_predicts, axis=0)) **2
y_error = np.nanmean((y - y_predicts) **2, axis=0)#.mean()
y_var = np.nanvar(y_predicts, axis=0, ddof = 1)

print(np.nanmean(y_bias) +  np.nanmean(y_var))

clf_type = "Decision tree"
print("{0}: {1:.4f} (error) = {2:.4f} (bias^2) "
          "+ {3:.4f} (var)".format(clf_type, np.nanmean(y_error), np.nanmean(y_bias), np.nanmean(y_var)))

print("{0}: {1:.4f} ((bias^2) + (var)) = {2:.4f} (bias^2) "
          "+ {3:.4f} (var)".format(clf_type, np.mean(y_bias) + np.mean(y_var), np.mean(y_bias), np.mean(y_var)))

In [ ]:
#clf = RandomForestRegressor(n_estimators=1000,max_depth=2)
clf = BayesianRidge(normalize=True)
scores = cross_val_score(clf, X, y, cv=10, scoring = make_scorer(mse))

dummy_clf = DummyRegressor(strategy='mean')
dummy_scores = cross_val_score(dummy_clf, X, y, cv=10, scoring = make_scorer(mse))

print("MSE: %0.8f +/- %0.8f" % (scores.mean(), scores.std()))
print("Dummy MSE: %0.8f +/- %0.8f" % (dummy_scores.mean(), dummy_scores.std()))
